In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

from scipy.stats import pearsonr
from transformers import CLIPSegProcessor, CLIPSegForImageSegmentation

clip_processor = CLIPSegProcessor.from_pretrained("CIDAS/clipseg-rd64-refined")
clip_model = CLIPSegForImageSegmentation.from_pretrained("CIDAS/clipseg-rd64-refined")

import os
import sys
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
from config import AGD20K_PATH, model_name

from VLM_model_dot_relative import MetricsTracker
from file_managing import (
    load_selected_samples,
    get_actual_path,
    get_gt_path,
    load_ground_truth,
    prompt_dict_obj,
    get_clipseg_heatmap,
    calculate_metrics,
    prompt_dict_obj
)

def min_max_normalize(arr):
    denom = arr.max() - arr.min()
    if denom == 0:
        return np.zeros_like(arr)
    return (arr - arr.min()) / (denom + 1e-8)



from transformers import Qwen3VLForConditionalGeneration, AutoProcessor
model_name= f"Qwen/Qwen3-VL-32B-Instruct"
processor = AutoProcessor.from_pretrained(model_name)
tok = processor.tokenizer

AGD20K_PATH

/home/bongo/anaconda3/envs/qwen3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


'/home/DATA/AGD20K'

In [2]:
import numpy as np

def check_heatmap_containment(heatmap_top, heatmap_obj, threshold=0.15, containment_ratio=0.8):
    """
    Args:
        containment_ratio (float): Top 영역의 몇 % 이상이 Obj와 겹쳐야 포함으로 볼 것인지 (기본 0.9 = 90%)
    """
    
    # 1. 텐서인 경우 numpy 변환
    if hasattr(heatmap_top, 'cpu'):
        heatmap_top = heatmap_top.detach().cpu().numpy()
    if hasattr(heatmap_obj, 'cpu'):
        heatmap_obj = heatmap_obj.detach().cpu().numpy()

    # 2. 이진 마스크 생성
    mask_top = heatmap_top > threshold
    mask_obj = heatmap_obj > threshold

    # 3. 면적 계산
    area_top = np.sum(mask_top)
    area_obj = np.sum(mask_obj)

    # 예외 처리: Top 히트맵이 아예 활성화되지 않은 경우 (면적 0)
    if area_top == 0:
        return False

    # 조건 1: Top의 면적이 Object 면적보다 작은가?
    is_smaller = area_top < area_obj
    
    # 4. 포함 관계 확인 (수정된 부분)
    # 교집합(Intersection) 영역 계산
    intersection = np.logical_and(mask_top, mask_obj)
    intersection_area = np.sum(intersection)

    # [수정됨] 교집합 면적이 Top 전체 면적의 90% 이상인지 확인
    # (intersection_area / area_top) >= 0.9 와 동일한 수식입니다.
    is_inside = intersection_area >= (area_top * containment_ratio)

    # 디버깅용: 실제 겹치는 비율 확인
    # print(f"Overlap Ratio: {intersection_area / area_top:.2f}")

    return is_smaller and is_inside

In [6]:
metrics_tracker_alloutput = MetricsTracker(name="all_output")
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

# 저장할 디렉토리 생성
Layername = "clipseg_top1"
output_dir = f"./output_{Layername}"  # 디렉토리 이름 변경 (구분 위해)s
os.makedirs(output_dir, exist_ok=True)

# Contrastive Subtraction 강도 설정 (0.5 ~ 1.0 추천)

POS_ALPHA = 0
results_list = []
pos_map = np.zeros((31, 31), dtype=np.float32)
for i in range(24):
    i += 1
    pkl_path = f"output_results/attention_result_full_output_32B_{i}.pkl"
    if not os.path.exists(pkl_path):
        continue
        
    df_output = pd.read_pickle(pkl_path)

    for idx, row in df_output.iterrows():
        object_name = row['object']
        action = row['action']
        filename = row['filename']
        output_description = row['output_sentence']
        output_attentions = row['output_attentions']
        PLSP_name = prompt_dict_obj[action][object_name]
        sum_heatmap = np.zeros((31, 31), dtype=np.float32)
        
        file_name_real = f"{AGD20K_PATH}/Seen/testset/egocentric/{action}/{object_name}/{filename}"
        gt_path = f"{AGD20K_PATH}/Seen/testset/GT/{action}/{object_name}/{filename.split('.')[0]}.png"
            
        # 이미지 로드
        if not os.path.exists(file_name_real):
            print(f"Image not found: {file_name_real}")
            continue

        orig_img = cv2.imread(file_name_real)
        orig_img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)
        h, w, _ = orig_img.shape

        print(f"[{idx}] Processing: {action} - {object_name}")

        # --- 1. CLIPSeg Mask 생성 ---

        clip_object_heatmap = get_clipseg_heatmap(
            file_name_real,
            clip_model,
            clip_processor,
            object_name,
            # PLSP_name,
        )

        # CLIPSeg 결과를 31x31로 리사이즈
        clip_heatmap_resized = cv2.resize(clip_object_heatmap, (31, 31), interpolation=cv2.INTER_LINEAR)
        clip_binary_mask = (clip_heatmap_resized > 0.15).astype(np.float32) # 필요시 사용

        # --- 2. [변경] Contrastive Attention Map 계산 ---
        token_scores = []
        token_idx = 0

        # token = output_attentions[-1]
        # # 토큰별 히트맵 초기화
        # token_heatmap = np.zeros((31, 31), dtype=np.float32)
        # token_head_count = 0
        
        # attention_value = token['attentions']
        # decoded_str = token['token_str'] # 디버깅용

        # for each_attention in attention_value:
        #     layer = each_attention['layer']
        #     head = each_attention['head']
        #     sum_heatmap += each_attention['heatmap']            
        #     token_heatmap += each_attention['heatmap']
        #     token_head_count += 1            
        
        # # 해당 레이어의 헤드가 하나도 없으면 스킵
        # if token_head_count == 0:
        #     raise ValueError("No valid tokens found.")

        # # Visual Dependency Score (S_img) 계산: 맵의 총합
        # s_img = token_heatmap.sum()
        # s_img_masked = (token_heatmap * clip_binary_mask).sum()
        
        # # 리스트에 저장
        # token_scores.append({
        #     "token": decoded_str,
        #     "token_idx" : token_idx,
        #     "score": s_img_masked,
        #     "score_ori": s_img,
        #     "heatmap": token_heatmap, 
        #     "count": token_head_count
        # })
        # token_idx +=1


        for token in output_attentions:
            # 토큰별 히트맵 초기화
            token_heatmap = np.zeros((31, 31), dtype=np.float32)
            token_head_count = 0
            
            attention_value = token['attentions']
            decoded_str = token['token_str'] # 디버깅용

            for each_attention in attention_value:
                layer = each_attention['layer']
                head = each_attention['head']
                sum_heatmap += each_attention['heatmap']            
                token_heatmap += each_attention['heatmap']
                token_head_count += 1            
            
            # 해당 레이어의 헤드가 하나도 없으면 스킵
            if token_head_count == 0:
                raise ValueError("No valid tokens found.")

            # Visual Dependency Score (S_img) 계산: 맵의 총합
            s_img = token_heatmap.sum()
            s_img_masked = (token_heatmap * clip_binary_mask).sum()
            
            # 리스트에 저장
            token_scores.append({
                "token": decoded_str,
                "token_idx" : token_idx,
                "score": s_img_masked,
                "score_ori": s_img,
                "heatmap": token_heatmap, 
                "count": token_head_count
            })
            token_idx +=1
        # 예외 처리: 토큰이 없을 경우
        if len(token_scores) == 0:
            raise ValueError("No valid tokens found.")

        # 정렬 (Score 기준 오름차순)
        sorted_tokens = sorted(token_scores, key=lambda x: x['score'])
        
        # Top 5 (Signal) & Bottom 5 (Noise) 선정
        # 토큰 개수가 10개 미만일 경우 처리
        num_select = min(1, len(sorted_tokens) // 2)
        if num_select < 1: num_select = 1 # 최소 1개

        top_tokens = sorted_tokens[-num_select:][::-1]   # Signal (명사, 핵심어)
        top_token_idx  = top_tokens[-1]['token_idx']
        top_token_text  = top_tokens[-1]['token']

        following_token_idx = top_token_idx + 1
        following_token = None
        for item in token_scores:
            if item['token_idx'] == following_token_idx:
                following_token = item
                break
        following_text = following_token['token']

        clip_top_heatmap = get_clipseg_heatmap(
            file_name_real,
            clip_model,
            clip_processor,
            top_token_text + ' ' + following_text,
            # PLSP_name
        )

        average_heatmap = sum_heatmap / token_head_count
        pos_map = average_heatmap.copy()
        # pos_map = np.sum([t['heatmap'] for t in top_tokens], axis=0)
        # pos_map /= len(top_tokens)
        if pos_map.max() > 0: pos_map /= pos_map.max()

        avg_norm = pos_map.copy()

        if check_heatmap_containment(clip_top_heatmap,clip_object_heatmap):
            clip_heatmap = clip_top_heatmap
            clipseg_input_text = top_token_text+ ' ' + following_text
            # Signal Map (Positive) 평균
        else:
            clip_heatmap = clip_object_heatmap
            clipseg_input_text = object_name

        print(f"Selected CLIP input : {clipseg_input_text}")
        # CLIPSeg 결과를 31x31로 리사이즈
        clip_heatmap_resized = cv2.resize(clip_heatmap, (31, 31), interpolation=cv2.INTER_LINEAR)
        clip_binary_mask = (clip_heatmap_resized > 0.15).astype(np.float32) # 필요시 사용

        
        # CLIPSeg와 Hadamard Product
        avg_norm_cliped = avg_norm * clip_heatmap_resized
        gamma =  0.75  # 0
        avg_norm_cliped = np.power(avg_norm_cliped, gamma)
        # 리사이즈 및 블러링
        avg_norm_cliped_rescaled = cv2.resize(avg_norm_cliped, (w, h), interpolation=cv2.INTER_LINEAR)
        
        sig = min(w, h) * 0.05
        k_val = int(sig * 3) * 2 + 1 
        kernel_size = (k_val, k_val)

        # 블러 적용
        blur_map = cv2.GaussianBlur(avg_norm_cliped_rescaled, kernel_size, sig)

        # 블러 후 다시 정규화
        blur_map = min_max_normalize(blur_map) # 함수가 정의되어 있다고 가정
        avg_norm_cliped_blur = blur_map
        
        # 시각화를 위해 31x31 맵도 원본 크기로 리사이즈
        avg_norm_resized_vis = cv2.resize(avg_norm, (w, h), interpolation=cv2.INTER_LINEAR)
        clip_vis = cv2.resize(clip_heatmap_resized, (w, h), interpolation=cv2.INTER_NEAREST)

        # --- 4. GT 평가 및 메트릭 계산 ---
        gt_map = load_ground_truth(gt_path) # 함수 정의 가정
        if gt_map is not None:
            metrics_dino = calculate_metrics(avg_norm_cliped_blur, gt_map) # 함수 정의 가정
            metrics_tracker_alloutput.update(metrics_dino) # 객체 정의 가정
            
            # 메트릭 텍스트
            metrics_text = f"[{object_name} {action}] KLD: {metrics_dino['KLD']:.4f} | SIM: {metrics_dino['SIM']:.4f} | NSS: {metrics_dino['NSS']:.4f}"
            metrics_tracker_alloutput.print_metrics(metrics_dino, filename)
        else:
            print("NO GT!!!")
            metrics_text = "No GT Available"
            continue
        
        results_list.append({
            'object': object_name,
            'action': action,
            'filename': filename,
            'output_sentence': output_description,
            'top_token_text': top_token_text,
            'following_text': following_text,
            'clip_input': clipseg_input_text,
            'KLD': metrics_dino['KLD'],
            'SIM': metrics_dino['SIM'],
            'NSS': metrics_dino['NSS']
        })

        # --- 5. 시각화 ---
        fig, axes = plt.subplots(1, 6, figsize=(24, 5)) # 사이즈 살짝 조정
        
        # Signal 단어와 Noise 단어 표시 (제목용)
        top_words = ",".join([f"'{t['token'].strip()}'" for t in top_tokens[:5]])
        
        main_title = f"Obj: {object_name} | Act: {action} |{metrics_text}\nTop Tokens: [{top_words}({top_token_idx } ),  clipseg input : {top_token_text} {following_text}] \n Whole answer : {output_description}"
        fig.suptitle(main_title, fontsize=14, fontweight='bold', y=0.98)

        # (1) 원본 이미지
        axes[0].imshow(orig_img)
        axes[0].set_title(f"Original\n({object_name})")
        axes[0].axis('off')

        # (2) Contrastive Attention (Pos - Neg)
        im1 = axes[1].imshow(avg_norm_resized_vis, cmap='jet', interpolation='bilinear')
        axes[1].set_title(f"Attention Map {Layername}")
        axes[1].axis('off')
        plt.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04)

        # (3) CLIPSeg Result
        axes[2].imshow(clip_vis, cmap='gray')
        axes[2].set_title(f"CLIPSeg {clipseg_input_text}")
        axes[2].axis('off')

        # (4) Hadamard (Contrastive x CLIPSeg)
        # 리사이즈하여 시각화
        hadamard_vis = cv2.resize(avg_norm_cliped, (w, h), interpolation=cv2.INTER_LINEAR)
        im3 = axes[3].imshow(hadamard_vis, cmap='jet', interpolation='bilinear')
        axes[3].set_title("Hadamard\n(Contrastive x CLIP)")
        axes[3].axis('off')
        plt.colorbar(im3, ax=axes[3], fraction=0.046, pad=0.04)

        # (5) Final Blurred Result
        im4 = axes[4].imshow(avg_norm_cliped_blur, cmap='jet', interpolation='bilinear')
        axes[4].set_title("Final Blurred")
        axes[4].axis('off')
        plt.colorbar(im4, ax=axes[4], fraction=0.046, pad=0.04)

        # (6) Ground Truth
        axes[5].imshow(gt_map, cmap='gray') # GT는 보통 binary 혹은 gray
        axes[5].set_title("Ground Truth")
        axes[5].axis('off')

        # 파일 저장
        save_path = os.path.join(output_dir, f"{object_name}_{action}_{filename.split('.')[0]}.png")
        plt.tight_layout()
        plt.savefig(save_path, bbox_inches='tight', dpi=150)
        plt.close(fig)

df_analy = pd.DataFrame(results_list)

[0] Processing: cut - apple


/home/bongo/anaconda3/envs/qwen3/lib/python3.9/site-packages/transformers/image_processing_utils.py:51: UserWarning: The following named arguments are not valid for `ViTImageProcessor.preprocess` and were ignored: 'padding'
  return self.preprocess(images, **kwargs)


Selected CLIP input : apple

Metrics for all_output apple_000054.jpg:
 all_output Current - KLD: 0.6967 | SIM: 0.6304 | NSS: 0.7173

Cumulative all_output  Averages over 1 samples:
Average - KLD: 0.6967 | SIM: 0.6304 | NSS: 0.7173

[1] Processing: eat - apple
Selected CLIP input :  flesh  of

Metrics for all_output apple_001541.jpg:
 all_output Current - KLD: 0.2812 | SIM: 0.7230 | NSS: 1.1925

Cumulative all_output  Averages over 2 samples:
Average - KLD: 0.4889 | SIM: 0.6767 | NSS: 0.9549

[2] Processing: peel - apple
Selected CLIP input :  colored  layer

Metrics for all_output apple_001541.jpg:
 all_output Current - KLD: 0.1693 | SIM: 0.7994 | NSS: 1.2102

Cumulative all_output  Averages over 3 samples:
Average - KLD: 0.3824 | SIM: 0.7176 | NSS: 1.0400

[3] Processing: hit - axe
Selected CLIP input :  blade  (

Metrics for all_output axe_000961.jpg:
 all_output Current - KLD: 0.8073 | SIM: 0.4638 | NSS: 1.3466

Cumulative all_output  Averages over 4 samples:
Average - KLD: 0.4886 |

/home/bongo/anaconda3/envs/qwen3/lib/python3.9/site-packages/transformers/image_processing_utils.py:51: UserWarning: The following named arguments are not valid for `ViTImageProcessor.preprocess` and were ignored: 'padding'
  return self.preprocess(images, **kwargs)


Selected CLIP input :  handle  (

Metrics for all_output badminton_racket_002255.jpg:
 all_output Current - KLD: 1.1181 | SIM: 0.3491 | NSS: 1.8696

Cumulative all_output  Averages over 6 samples:
Average - KLD: 0.5826 | SIM: 0.5996 | NSS: 1.4789

[6] Processing: swing - badminton_racket
Selected CLIP input : inton  racket

Metrics for all_output badminton_racket_003649.jpg:
 all_output Current - KLD: 4.5053 | SIM: 0.0174 | NSS: -0.4329

Cumulative all_output  Averages over 7 samples:
Average - KLD: 1.1430 | SIM: 0.5164 | NSS: 1.2058

[7] Processing: cut - banana
Selected CLIP input : banana

Metrics for all_output banana_002623.jpg:
 all_output Current - KLD: 0.2136 | SIM: 0.7491 | NSS: 1.4466

Cumulative all_output  Averages over 8 samples:
Average - KLD: 1.0268 | SIM: 0.5455 | NSS: 1.2359

[8] Processing: eat - banana
Selected CLIP input : banana

Metrics for all_output banana_002458.jpg:
 all_output Current - KLD: 0.1017 | SIM: 0.8471 | NSS: 1.7448

Cumulative all_output  Averages 

...

Selected CLIP input :  black  circular

Metrics for all_output surfboard_002422.jpg:
 all_output Current - KLD: 2.4812 | SIM: 0.1191 | NSS: -0.3527

Cumulative all_output  Averages over 106 samples:
Average - KLD: 1.0629 | SIM: 0.4534 | NSS: 1.2660

[106] Processing: hold - surfboard
Selected CLIP input : surfboard

Metrics for all_output surfboard_002631.jpg:
 all_output Current - KLD: 0.7549 | SIM: 0.4836 | NSS: 2.6717

Cumulative all_output  Averages over 107 samples:
Average - KLD: 1.0600 | SIM: 0.4537 | NSS: 1.2791

[107] Processing: jump - surfboard
Selected CLIP input : surfboard

Metrics for all_output surfboard_000658.jpg:
 all_output Current - KLD: 0.9082 | SIM: 0.4388 | NSS: 0.3362

Cumulative all_output  Averages over 108 samples:
Average - KLD: 1.0586 | SIM: 0.4535 | NSS: 1.2704

[108] Processing: lie_on - surfboard
Selected CLIP input : surfboard

Metrics for all_output surfboard_000221.jpg:
 all_output Current - KLD: 0.1024 | SIM: 0.8554 | NSS: 2.3661

Cumulative all_out

/home/bongo/anaconda3/envs/qwen3/lib/python3.9/site-packages/transformers/image_processing_utils.py:51: UserWarning: The following named arguments are not valid for `ViTImageProcessor.preprocess` and were ignored: 'padding'
  return self.preprocess(images, **kwargs)


Selected CLIP input : tennis_racket

Metrics for all_output tennis_racket_002268.jpg:
 all_output Current - KLD: 3.4743 | SIM: 0.0601 | NSS: -0.4287

Cumulative all_output  Averages over 111 samples:
Average - KLD: 1.0639 | SIM: 0.4564 | NSS: 1.2559

[111] Processing: hold - tennis_racket
Selected CLIP input :  handle  of

Metrics for all_output tennis_racket_001785.jpg:
 all_output Current - KLD: 0.6092 | SIM: 0.5482 | NSS: 3.2809

Cumulative all_output  Averages over 112 samples:
Average - KLD: 1.0599 | SIM: 0.4572 | NSS: 1.2739

[112] Processing: swing - tennis_racket
Selected CLIP input :  handle  of

Metrics for all_output tennis_racket_003066.jpg:
 all_output Current - KLD: 1.1358 | SIM: 0.3532 | NSS: 1.8164

Cumulative all_output  Averages over 113 samples:
Average - KLD: 1.0605 | SIM: 0.4563 | NSS: 1.2787

[113] Processing: brush_with - toothbrush
Selected CLIP input :  at  the

Metrics for all_output toothbrush_001764.jpg:
 all_output Current - KLD: 2.2966 | SIM: 0.1348 | NSS:

/home/bongo/anaconda3/envs/qwen3/lib/python3.9/site-packages/transformers/image_processing_utils.py:51: UserWarning: The following named arguments are not valid for `ViTImageProcessor.preprocess` and were ignored: 'padding'
  return self.preprocess(images, **kwargs)


Selected CLIP input : toothbrush

Metrics for all_output toothbrush_001991.jpg:
 all_output Current - KLD: 1.2360 | SIM: 0.3178 | NSS: 1.8808

Cumulative all_output  Averages over 116 samples:
Average - KLD: 1.0782 | SIM: 0.4503 | NSS: 1.2805

[116] Processing: drink_with - wine_glass
Selected CLIP input :  bowl  of

Metrics for all_output wine_glass_003343.jpg:
 all_output Current - KLD: 1.5623 | SIM: 0.2401 | NSS: 0.9313

Cumulative all_output  Averages over 117 samples:
Average - KLD: 1.0823 | SIM: 0.4485 | NSS: 1.2775

[117] Processing: hold - wine_glass
Selected CLIP input :  its  stem

Metrics for all_output wine_glass_002374.jpg:
 all_output Current - KLD: 1.1163 | SIM: 0.3738 | NSS: 1.2268

Cumulative all_output  Averages over 118 samples:
Average - KLD: 1.0826 | SIM: 0.4479 | NSS: 1.2770

[118] Processing: pour - wine_glass
Selected CLIP input : wine_glass

Metrics for all_output wine_glass_000186.jpg:
 all_output Current - KLD: 1.3131 | SIM: 0.3126 | NSS: 0.9142

Cumulative a

In [ ]:
# 민맥스 안하고 그냥 맥스분에일한거
LAST-cvpr25 - KLD: 3.2672 | SIM: 0.1017 | NSS: -0.1982
LAST    - KLD:  KLD: 1.2582 | SIM: 0.4002 | NSS: 1.0195
ALL     - KLD:  KLD: 1.0893 | SIM: 0.4449 | NSS: 1.2702 //  KLD: 1.1319 | SIM: 0.4425 | NSS: 1.1987
TOP1 O only : : 1.0659 | SIM: 0.4494 | NSS: 1.3036
TOP1    - KLD: 1.0486 | SIM: 0.4630 | NSS: 1.3187  
TOP1_GT - KLD: 0.9934 | SIM: 0.4644 | NSS: 1.3921


In [ ]:
TOP 90 : Average - KLD: 1.1244 | SIM: 0.4453 | NSS: 1.2122
TOP 95 : Average - KLD: 1.1112 | SIM: 0.4485 | NSS: 1.2430
TOP 99 : KLD: 1.0911 | SIM: 0.4547 | NSS: 1.2855
TOP 99 Bottom 50 : KLD: 1.0911 | SIM: 0.4547 | NSS: 1.2855
TOP 80 Bottom 50 :KLD: 1.1480 | SIM: 0.4391 | NSS: 1.1538
ALL_token_exp075 :KLD: 1.2934 | SIM: 0.3905 | NSS: 0.9825
TOP1 : 1.0901 | SIM: 0.4564 | NSS: 1.2929
TOP1_exp0.75 : KLD: 1.0885 | SIM: 0.4532 | NSS: 1.2754
CLIPSEG_TOP1_exp0.75 : 1.0672 | SIM: 0.4499 | NSS: 1.3048
CLIPSEG_TOP12_exp0.75 : KLD: 1.0653 | SIM: 0.4484 | NSS: 1.3006
CLIPSEG_TOP123_exp0.75 :  1.0725 | SIM: 0.4469 | NSS: 1.2891

CLIPSEG_TOP1_GT_exp0.75 :  KLD: 0.9955 | SIM: 0.4649 | NSS: 1.3919



TOP1_real_exp0.5 : KLD: 1.1299 | SIM: 0.4168 | NSS: 1.3019
CLIP_fillter top1 :  KLD: 1.0788 | SIM: 0.4702 | NSS: 1.2851
CLIP_fillter top1+following :  KLD: 1.0782 | SIM: 0.4732 | NSS: 1.2873
CLIP_fillter top1+following_exp0.3 : KLD: 1.0730 | SIM: 0.4661 | NSS: 1.2646
CLIP_fillter top1+following_exp0.75 :  KLD: 1.0625 | SIM: 0.4734 | NSS: 1.2900
CLIP_fillter top1+following_exp0.75_real :  KLD: 1.0474 | SIM: 0.4607 | NSS: 1.3293
CLIP_fillter top1+following_exp0.75_real_namable :  KLD: 1.1120 | SIM: 0.4399 | NSS: 1.2498
CLIP_fillter top1+following_ININ_exp0.75_real :  KLD: 1.0567 | SIM: 0.4571 | NSS: 1.3125
CLIP_fillter top1+following_exp0.5_real :  KLD: 1.0970 | SIM: 0.4276 | NSS: 1.3465
CLIP_fillter top1+following_exp2 :  Average - KLD: 1.5235 | SIM: 0.4005 | NSS: 0.9573

CLIP_fillter PLSP_exp0.75_real :  KLD: 1.0015 | SIM: 0.4608 | NSS: 1.4040